In [ ]:
def portfolio_variance(w, C):
    return np.dot(w.T, np.dot(C, w))

def gradient(C, w):
    return 2 * np.dot(C, w)

def projection_onto_simplex(w):
    """Project vector w onto the simplex defined by the constraints w >= 0 and sum(w) = 1."""
    if np.sum(w) > 1:
        w = w / np.sum(w)  # Scale if exceeding 1
    return np.maximum(w, 0)  # Ensure non-negativity

def barrier_method(C, initial_w, mu_start=10, mu_end=1e-4, rho=0.5, epsilon=1e-8, max_iter=100):
    n = len(initial_w)
    w = initial_w.copy()
    mu = mu_start

    for iter in range(max_iter):
        w_proj = projection_onto_simplex(w)  # Ensure non-negative weights and sum to 1
        
     
        inv_w_proj = 1 / (w_proj.flatten() + 1e-10)  # Inverse for log barrier with small epsilon
    
        if len(w_proj) != C.shape[0]:
            raise ValueError("Weight vector length must match the covariance matrix dimensions.")
        
        gradient_value = gradient(C, w_proj.flatten()) - mu * np.sum(inv_w_proj) * np.ones(n)

        step_size = 1.0
        while True:
            new_w = w - step_size * gradient_value
            new_w_proj = projection_onto_simplex(new_w)  # Project to ensure feasible weights
            if np.all(new_w_proj >= 0):  # Ensure non-negative weights
                break
            step_size *= rho 
        # Update weights and reduce mu
        w = new_w
        mu *= 0.5  # Reduce barrier parameter

        # Check for convergence
        if np.linalg.norm(gradient_value) < epsilon:
            print(f"Converged after {iter} iterations.")
            break

    return w

# Example usage
C = np.cov(HSi_train, rowvar=False)  # Ensure rowvar=False if columns are assets


print("Shape of C:", C.shape) 
print("Shape of HSi_train:", HSi_train.shape)  

# Initialize weights based on the number of assets
num_assets = C.shape[0] 
w_init = (1 / num_assets) * np.ones(num_assets)


w_init /= np.sum(w_init)

optimal_weights_adv, w_unconstrained_history = gradient_descent_with_projection(C, w_init, learning_rate=0.01, max_iter=1000)

# Output the results
print(f"Optimal Weights: {optimal_weights_adv}")
print(f"Final Portfolio Variance: {portfolio_variance(optimal_weights_adv, C)}")

# Plotting the results for w_constrained
plt.figure(figsize=(6, 4))
plt.plot(w_constrained, marker='o', label='Constrained Weights (Barrier Method)')
plt.title('Constrained Optimal Weights (Barrier Method)')
plt.xlabel('Asset Index')
plt.ylabel('Weight Value')
plt.grid()
plt.legend()
plt.show()